In [12]:
import os
import numpy as np
import h5py
import glob
import cv2
from PIL import Image
from keras.models import load_model
from keras.layers import Lambda, Input
import h5py

In [ ]:
def get_triplets(data, labels):
    pos_label, neg_label = np.random.choice(labels, 2, replace=False)
    pos_indexes = np.where(labels == pos_label)[0]
    neg_indexes = np.where(labels == neg_label)[0]
    np.random.shuffle(pos_indexes)
    np.random.shuffle(neg_indexes)
    anchor = data[pos_indexes[0]]
    positive = data[pos_indexes[-1]]
    negative = data[neg_indexes[0]]
    return (anchor, positive, negative)

In [ ]:
def dump_features(image_paths, labels, hdf5_path, feature_extractor, image_formats=("jpg")):
    db = h5py.File(hdf5_path, mode="w")
    features_shape = ((len(labels),), feature_extractor.output_shape[1:])
    features_shape = [dim for sublist in features_shape for dim in sublist]
    imageIDDB = db.create_dataset("image_ids", shape=(len(labels),), 
        dtype=h5py.special_dtype(vlen=unicode))
    featuresDB = db.create_dataset("features", 
        shape=features_shape, dtype="float")
    labelsDB = db.create_dataset("labels", 
        shape=(len(labels),), dtype="int")
    for i in range(0, len(labels), 32):
        start,end = i, i+32
        image_ids = [path.split("/")[-1] for path in image_paths[start:end]]
        images = [cv2.imread(path,1) for path in image_paths[start:end]]
        features = feature_extractor.extract(images)
        imageIDDB[start:end] = image_ids
        featuresDB[start:end] = features
        labelsDB[start:end] = labels[start:end]
        print "Extracting {}/{}".format(i+1+32, len(labels))
    db.close()

In [ ]:
def extract_features(hdf5_path):
    db = h5py.File(hdf5_path,mode="r")
    features = db["features"][:]
    labels = db["labels"][:]
    return (features, labels)

def extract_embeddings(features, model):
    embeddings = model.predict([features, features, features])
    return embeddings[:,:,0]

In [8]:
data=[]
labels=[]
train_data_dir = 'C:/Users/hp/Downloads/data/train/'
cat_root = train_data_dir + 'cats' + '/'
cats=os.listdir(cat_root)

root_list = os.listdir(cat_root)
for cat in cats:
    imag=cv2.imread(cat_root + cat)
    img_from_ar = Image.fromarray(imag, 'RGB')
    resized_image = img_from_ar.resize((50, 50))
    data.append(np.array(resized_image))
    labels.append(0)

dog_root = train_data_dir + 'dogs' + '/'
dogs=os.listdir(dog_root)
for dog in dogs:
    imag=cv2.imread(dog_root + dog)
    img_from_ar = Image.fromarray(imag, 'RGB')
    resized_image = img_from_ar.resize((150, 150))
    data.append(np.array(resized_image))
    labels.append(1)
    
# birds=os.listdir("birds")
# for bird in birds:
#     imag=cv2.imread("birds/"+bird)
#     img_from_ar = Image.fromarray(imag, 'RGB')
#     resized_image = img_from_ar.resize((50, 50))
#     data.append(np.array(resized_image))
#     labels.append(2)
# fishs=os.listdir("fishs")
# for fish in fishs:
#     imag=cv2.imread("fishs/"+fish)
#     img_from_ar = Image.fromarray(imag, 'RGB')
#     resized_image = img_from_ar.resize((50, 50))
#     data.append(np.array(resized_image))
#     labels.append(3)

In [11]:
animals=np.array(data)
labels=np.array(labels)

In [13]:
labels

array([0, 0, 0, ..., 1, 1, 1])

In [45]:
(np.arange(1000)/4)+1

array([  1.  ,   1.25,   1.5 ,   1.75,   2.  ,   2.25,   2.5 ,   2.75,
         3.  ,   3.25,   3.5 ,   3.75,   4.  ,   4.25,   4.5 ,   4.75,
         5.  ,   5.25,   5.5 ,   5.75,   6.  ,   6.25,   6.5 ,   6.75,
         7.  ,   7.25,   7.5 ,   7.75,   8.  ,   8.25,   8.5 ,   8.75,
         9.  ,   9.25,   9.5 ,   9.75,  10.  ,  10.25,  10.5 ,  10.75,
        11.  ,  11.25,  11.5 ,  11.75,  12.  ,  12.25,  12.5 ,  12.75,
        13.  ,  13.25,  13.5 ,  13.75,  14.  ,  14.25,  14.5 ,  14.75,
        15.  ,  15.25,  15.5 ,  15.75,  16.  ,  16.25,  16.5 ,  16.75,
        17.  ,  17.25,  17.5 ,  17.75,  18.  ,  18.25,  18.5 ,  18.75,
        19.  ,  19.25,  19.5 ,  19.75,  20.  ,  20.25,  20.5 ,  20.75,
        21.  ,  21.25,  21.5 ,  21.75,  22.  ,  22.25,  22.5 ,  22.75,
        23.  ,  23.25,  23.5 ,  23.75,  24.  ,  24.25,  24.5 ,  24.75,
        25.  ,  25.25,  25.5 ,  25.75,  26.  ,  26.25,  26.5 ,  26.75,
        27.  ,  27.25,  27.5 ,  27.75,  28.  ,  28.25,  28.5 ,  28.75,
      

In [48]:
image_formats=("jpg","png")
images_dir = 'C:/Users/hp/Downloads/data/train/cats/'
image_paths = []

for image_format in image_formats:
    image_paths += glob.glob("{}/*.{}".format(images_dir, image_format))

In [49]:
image_paths

['C:/Users/hp/Downloads/data/train/cats\\cat.0.jpg',
 'C:/Users/hp/Downloads/data/train/cats\\cat.1.jpg',
 'C:/Users/hp/Downloads/data/train/cats\\cat.10.jpg',
 'C:/Users/hp/Downloads/data/train/cats\\cat.100.jpg',
 'C:/Users/hp/Downloads/data/train/cats\\cat.1000.jpg',
 'C:/Users/hp/Downloads/data/train/cats\\cat.1001.jpg',
 'C:/Users/hp/Downloads/data/train/cats\\cat.1002.jpg',
 'C:/Users/hp/Downloads/data/train/cats\\cat.1003.jpg',
 'C:/Users/hp/Downloads/data/train/cats\\cat.1004.jpg',
 'C:/Users/hp/Downloads/data/train/cats\\cat.1005.jpg',
 'C:/Users/hp/Downloads/data/train/cats\\cat.1006.jpg',
 'C:/Users/hp/Downloads/data/train/cats\\cat.1007.jpg',
 'C:/Users/hp/Downloads/data/train/cats\\cat.1008.jpg',
 'C:/Users/hp/Downloads/data/train/cats\\cat.1009.jpg',
 'C:/Users/hp/Downloads/data/train/cats\\cat.101.jpg',
 'C:/Users/hp/Downloads/data/train/cats\\cat.1010.jpg',
 'C:/Users/hp/Downloads/data/train/cats\\cat.1011.jpg',
 'C:/Users/hp/Downloads/data/train/cats\\cat.1012.jpg',
 '

In [50]:
image_path_list

['C:/Users/hp/Downloads/data/train/cats/cat.0.jpg',
 'C:/Users/hp/Downloads/data/train/cats/cat.1.jpg',
 'C:/Users/hp/Downloads/data/train/cats/cat.10.jpg',
 'C:/Users/hp/Downloads/data/train/cats/cat.100.jpg',
 'C:/Users/hp/Downloads/data/train/cats/cat.1000.jpg',
 'C:/Users/hp/Downloads/data/train/cats/cat.1001.jpg',
 'C:/Users/hp/Downloads/data/train/cats/cat.1002.jpg',
 'C:/Users/hp/Downloads/data/train/cats/cat.1003.jpg',
 'C:/Users/hp/Downloads/data/train/cats/cat.1004.jpg',
 'C:/Users/hp/Downloads/data/train/cats/cat.1005.jpg',
 'C:/Users/hp/Downloads/data/train/cats/cat.1006.jpg',
 'C:/Users/hp/Downloads/data/train/cats/cat.1007.jpg',
 'C:/Users/hp/Downloads/data/train/cats/cat.1008.jpg',
 'C:/Users/hp/Downloads/data/train/cats/cat.1009.jpg',
 'C:/Users/hp/Downloads/data/train/cats/cat.101.jpg',
 'C:/Users/hp/Downloads/data/train/cats/cat.1010.jpg',
 'C:/Users/hp/Downloads/data/train/cats/cat.1011.jpg',
 'C:/Users/hp/Downloads/data/train/cats/cat.1012.jpg',
 'C:/Users/hp/Downlo

In [2]:
image_path_list=[]
labels=[]
train_data_dir = 'C:/Users/hp/Downloads/data/train/'
cat_root = train_data_dir + 'cats' + '/'
cats=os.listdir(cat_root)

for cat in cats:
    image_path_list.append(cat_root + cat)
    labels.append(0)

dog_root = train_data_dir + 'dogs' + '/'
dogs=os.listdir(dog_root)
for dog in dogs:
    image_path_list.append(dog_root + dog)
    labels.append(1)
    
labels = np.array(labels)

In [4]:
labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [42]:
labels
pos_label, neg_label = np.random.choice(labels, 2, replace=False)
#print('positive:' + pos_label + '   negative:' + neg_label)

In [51]:
range(0, len(labels), 32)

range(0, 1200, 32)

In [6]:
hdf5_path = 'C:/Users/hp/Downloads/data/similarity/similarity_db.hdf5'
db = h5py.File(hdf5_path, mode="w")

imageIDDB = db.create_dataset("image_ids", shape=(len(labels),),
                                  dtype=h5py.special_dtype(vlen=np.unicode))

labelsDB = db.create_dataset("labels",
                                 shape=(len(labels),), dtype="int")

for i in range(0, len(labels), 16):
        start,end = i, i+16
        image_ids = [path.split("/")[-1] for path in image_path_list[start:end]]
        images = [cv2.imread(path,1) for path in image_path_list[start:end]]
        #features = feature_extractor.extract(images)
        imageIDDB[start:end] = image_ids
        #featuresDB[start:end] = features
        labelsDB[start:end] = labels[start:end]
        print("Extracting {}/{}".format(i+1+16, len(labels)))

Extracting 33/1200
Extracting 65/1200
Extracting 97/1200
Extracting 129/1200
Extracting 161/1200
Extracting 193/1200
Extracting 225/1200
Extracting 257/1200
Extracting 289/1200
Extracting 321/1200
Extracting 353/1200
Extracting 385/1200
Extracting 417/1200
Extracting 449/1200
Extracting 481/1200
Extracting 513/1200
Extracting 545/1200
Extracting 577/1200
Extracting 609/1200
Extracting 641/1200
Extracting 673/1200
Extracting 705/1200
Extracting 737/1200
Extracting 769/1200
Extracting 801/1200
Extracting 833/1200
Extracting 865/1200
Extracting 897/1200
Extracting 929/1200
Extracting 961/1200
Extracting 993/1200
Extracting 1025/1200
Extracting 1057/1200
Extracting 1089/1200
Extracting 1121/1200
Extracting 1153/1200
Extracting 1185/1200
Extracting 1217/1200


In [10]:
def extract_features(hdf5_path):
    db = h5py.File(hdf5_path,mode="r")
    features = db["image_ids"][:]
    labels = db["labels"][:]

    return (features, labels)

In [11]:
features, labels = extract_features('C:/Users/hp/Downloads/data/similarity/similarity_db.hdf5')

In [16]:
len(features)

1200

In [19]:
model = load_model('C:/Users/hp/Downloads/data/model/top_model_vgg/final_model.h5')

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [3]:
np.array([int('cats'!= 'cats')],dtype=np.float32)

array([0.], dtype=float32)

In [5]:
input_shape=(None, 4, 4, 512)

In [8]:
input_shape[1:]

(4, 4, 512)

In [11]:
Input(input_shape[1:])

<tf.Tensor 'input_1:0' shape=(?, 4, 4, 512) dtype=float32>

In [13]:
def extract_features(hdf5_path):
    db = h5py.File(hdf5_path,mode="r")
    features = db["features"][:]
    labels = db["labels"][:]

    return (features, labels)

In [14]:
features, labels = extract_features('C:/Users/hp/Downloads/data/similarity/similarity_db.hdf5')
features.shape

(1200, 4, 4, 512)